## Load packages

In [13]:
import os, sys, glob, scipy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tqdm

## Learning Model

The learning model used to generate choices is a simple Rescorla-Wagner (Rescorla & Wagner, 1972) model.

$$ Q(action, trial) = Q(action, trial - 1) + \alpha*prediction_{error}$$
$$ prediction_{error} = r(trial-1) - Q(action, trial - 1)$$
$$ 0 <= \alpha <= 1 $$

In the previous scripts, you have already read about the RL model and also already came across the softmax choice rule (recall the orange juice/coffee example). Now, we are going to use the softmax function again to generate choice probabilities.

### Softmax

In [14]:
def softmax(q_values, beta):
    # Inputs:
    #       q_values: value of different actions, often just 2
    #       beta: inverse temperature, also known as exploitation parameter
    # Outputs:
    #       choice_probs: model's predicted probability of actions

    # Numerator represents value of utility of single choice
    numerator = np.exp(np.multiply(q_values, beta))

    # Denominator represents sum of total value of utilities
    denominator = np.sum(np.exp(np.multiply(q_values, beta)))
    
    # Outputs
    choice_probs = numerator / denominator
        
    return choice_probs


## Simulation

The function task_parameters generates the experimental data that an agent will receive. It defines the volatility of the switching, the block number, the trial number, the probability that the left and right levers give rewards, and whether they actually would give a reward (random sample drawn from that probability).

In [15]:
def task_parameters(volatility = 'low'): 
    
    # Generate dataframe to store simulated results
    sim_exp = pd.DataFrame(columns = ['volatility', 'block', 'block_trial', 'exp_trial', 
                                       'prob_reward_left', 'prob_reward_right', 'reward_left', 'reward_right'])
    
    # Trial sequence
    block_seq = [[17, 15, 19, 18, 15, 17, 19, 15], # high volatility
                [35, 30, 35, 35]] # low volatility
    reward_probs_left = [.7, .3] 
    reward_probs_right = [.3, .7] 

    if volatility == 'low':
        task_blocks = block_seq[1]
        block_prob_reward_left = np.array(reward_probs_left*(np.int(len(task_blocks)/2)))
        block_prob_reward_right = np.array(reward_probs_right*(np.int(len(task_blocks)/2)))
    else: 
        task_blocks = block_seq[0]
        block_prob_reward_left = np.array(reward_probs_left*(np.int(len(task_blocks)/2)))
        block_prob_reward_right = np.array(reward_probs_right*(np.int(len(task_blocks)/2)))

    # generate trials
    trial = 0
    for block in range(len(task_blocks)): 
        block_trial = 0
        # Loop through trials of the block
        for block_trial in range(task_blocks[block]):
            trial += 1
            block_trial += 1
            
            # Generate reward for both choices
            prob_reward_left = block_prob_reward_left[block] # pulls correct probability for left
            prob_reward_right = block_prob_reward_right[block] # pulls correct probability for right
            
            # Left choice
            if np.random.rand() <= prob_reward_left:
                reward_left = 1
            else: 
                reward_left = 0
                
            # Right choice
            if np.random.rand() <= prob_reward_right:
                reward_right = 1
            else: 
                reward_right = 0
                
            # Store results
            trial_data = pd.DataFrame([[volatility, block+1, block_trial, trial, 
                                        prob_reward_left, prob_reward_right, reward_left, reward_right]], 
                                     columns = ['volatility', 'block', 'block_trial', 'exp_trial', 
                                       'prob_reward_left', 'prob_reward_right', 'reward_left', 'reward_right'])
            sim_exp = sim_exp.append(trial_data).reset_index(drop=True)
            
    return sim_exp


### Reinforcement Learning

With the simulated task parameters from the task_parameter function, you can now generate choices with rl_simulate. 

In [16]:
def rl_simulate(params, sim_exp): 
    data = sim_exp.copy()
    # Initialize parameters
    # two alphas? (positive vs negative pes), stickiness? (likelihood of just repeating past action)
    # two-step modeling
    alpha = params[0] # learning param
    beta = params[1] # exploitation param
    q_values = [.5, .5] # q-values for left and right choices start at .5
    
    # Loop over rows in data
    for index, row in data.iterrows(): # loop over rows, slow but easy to understand and index
        
        # Compute probability of chosing left or right
        choice_probs = softmax(q_values, beta) 
        
        # Make choice weighted by probabilities
        # probabilities given by left, right
        if np.random.rand() <= choice_probs[0]:
            choice = 'left'
            choice_num = 0
        else:
            choice = 'right'
            choice_num = 1

                    
        # Store (keep above update)      
        data.at[index, 'choice'] = choice # useful way to append single values
        data.at[index, 'choice_num'] = choice_num
        data.at[index, 'prob_left'] = choice_probs[0]
        data.at[index, 'prob_right'] = choice_probs[1]
        data.at[index, 'q_left'] = q_values[0]
        data.at[index, 'q_right'] = q_values[1]
        
        # Update q_values (to be used on next trial)
        # All the magic happens here
        if choice == 'left':
            pe = row['reward_left'] - q_values[choice_num] # compute pe = reward - q_value['choice']
            q_values[choice_num] = q_values[choice_num] + alpha*pe # update value
        else:
            pe = row['reward_right'] - q_values[choice_num] # compute pe
            q_values[choice_num] = q_values[choice_num] + alpha*pe # update value
                
    # Add final information
    data['alpha'] = alpha
    data['beta'] = beta
    
    return data

##### Exercise 1
Now you have a function to simulate task parameters and a function to simulate choice. Using these two functions, visualize the probability of getting a reward for choosing the right lever and the simulated choice. 
You can also add a running average to visualize how the simulated subject behaved on average. 

# Fit simulated data

##### Exercise 2

Adjust rl_simulate to rl_simulate_fit. The rl_simulate_fit function returns the negative loglikelihood based on the alpha and beta parameters and some input data.

In [ ]:
def rl_simulate_fit(params, data): 
    

    
    
    
    
    
    
                 
    return -log_likelihood

## Parameter fitting

##### Exercise 3

1. Define your true parameters
2. simulate task parameters
3. simulate data
4. using the function you defined in exercise 2, fit the model to the simulated data to obtain the best parameters
5. Repeat the model fitting 20 times and store results in fit_results

You can draw inspiration from notebook 4